# Plotting eReefs model results with Bokeh

This notebook shows how to draw plots of datasets using Bokeh as the backend. It is adapted from the similar notebook in the [emsarray-notebooks repository](https://github.com/csiro-coasts/emsarray-notebooks/blob/master/bokeh.ipynb).

---


In [ ]:
import emsarray
import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap


output_notebook()


def wgs84_to_web_mercator(data, lon="lons", lat="lats"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    data["xs"] = [r * (k * np.pi/180.0) for r in data["lons"]]
    data["ys"] = [np.log(np.tan((90 + r) * np.pi/360.0)) * k for r in data["lats"]]
    return data


In [ ]:


ds = emsarray.open_dataset("https://thredds.nci.org.au/thredds/dodsC/fx3/model_data/gbr4_2.0.ncml?zc[0:1:46],longitude[0:1:179][0:1:599],latitude[0:1:179][0:1:599],botz[0:1:179][0:1:599],time[0:1:10],temp[0:1:10][0:1:46][0:1:179][0:1:599]")
ds


In [ ]:
temp = ds.ems.ravel(ds['temp'].isel(time=1, k=-1))[ds.ems.mask]

# Convert the dataet polygons and the scalar values in to a structure that bokeh can use
polygons = ds.ems.polygons[ds.ems.mask]
polygon_coords = [np.asarray(p.exterior.coords) for p in polygons]
data = wgs84_to_web_mercator({
    'lons': [pc[:, 0] for pc in polygon_coords],
    'lats': [pc[:, 1] for pc in polygon_coords],
    'temp': temp.values,
})

# Make a Bokeh figure
fig = figure(x_axis_type="mercator", y_axis_type="mercator", frame_width=800, frame_height=600)
fig.add_tile(tile_source="CartoDB Positron")

# # Add the dataset polygons
colour = LinearColorMapper(Viridis256, low=np.nanmin(temp.values), high=np.nanmax(temp.values), nan_color=(0, 0, 0, 0))
fig.patches(
    'xs', 'ys', source=data,
    fill_color={'field': 'temp', 'transform': colour},
    line_color={'field': 'temp', 'transform': colour})

# Add a colour bar
cb = ColorBar(color_mapper=colour, title="Sea surface temperature")
fig.add_layout(cb, 'right')

# Done!
show(fig)


We can also save this output as a .html page with the bokeh library using code like:

```python
import bokeh.io
import bokeh.resources
bokeh.io.save(fig, filename='Assets/test_bokeh_plot.html', title='GBR4_H2p0 Sea Surface Temperature', resources=bokeh.resources.CDN)

```

but that created a positively enormous file in our tests, so we've disabled it here.

Similarly, if you are working in an environment where you can install `selenium` and either `geckodriver` for Firefox or `ChromeDriver` for Chromium then you can export your Bokeh plot
to a PNG image using code like:

```python
import bokeh.io
bokeh.io.export_png(fig, filename='Assets/test_bokeh_plot.png')
```

As we were not prepared to try to run a web browser in our Jupyter notebooks, we leave saving these plots as an exercise for the reader.

You can find out more about exporting Bokeh plots [here](https://docs.bokeh.org/en/latest/docs/user_guide/output/export.html)